In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
#df_gscore = pd.read_csv('data/genome-scores.csv')
#df_gtags = pd.read_csv('data/genome-tags.csv')
#df_links = pd.read_csv('data/links.csv')
#df_movies = pd.read_csv('data/movies.csv')
df_rating = pd.read_csv('data/ratings.csv')
#df_tagspd = pd.read_csv('data/tags.csv')

# Interpreting the model

In [86]:
user_mean_vector = {}

for index, row in watched_movies_grouped.iterrows():
    userId = row['userId']
    given_movies_ids = row['watched_movies']
    given_movies_ids = [item for item in given_movies_ids if item in df.index.tolist()]
    user_mean_vector[userId] = df.loc[given_movies_ids].mean()

user_mean_vector

{170: 1           0.042530
 2           0.043805
 3           0.086430
 4           0.116049
 5           0.191616
               ...   
 Romance     0.162562
 Sci-Fi      0.162562
 Thriller    0.251232
 War         0.044335
 Western     0.009852
 Length: 1149, dtype: float64,
 984: 1           0.068367
 2           0.055908
 3           0.076318
 4           0.074163
 5           0.122991
               ...   
 Romance     0.236453
 Sci-Fi      0.211823
 Thriller    0.285714
 War         0.039409
 Western     0.024631
 Length: 1149, dtype: float64,
 1061: 1           0.062171
 2           0.057084
 3           0.052452
 4           0.064071
 5           0.105651
               ...   
 Romance     0.083744
 Sci-Fi      0.714286
 Thriller    0.305419
 War         0.009852
 Western     0.014778
 Length: 1149, dtype: float64,
 1317: 1           0.062218
 2           0.051314
 3           0.067698
 4           0.085770
 5           0.142198
               ...   
 Romance     0.147783
 Sci-

In [2]:
df_rating.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 610.4 MB


# modifying get_df() so it has tag names instead of tagId

In [2]:
from data_script.Preprocess_Content_Based import get_releaseyear
def get_df():

    
    # Importing the genomes score and genome tags data sets as pandas.DataFrame
    df_gscore = pd.read_csv('data/genome-scores.csv')
    df_gtags = pd.read_csv('data/genome-tags.csv')

    # Creating the genome dataset

    df_gtagscore = df_gtags.merge(df_gscore, how='right', on='tagId').drop('tagId', axis=1)
    df_gtagscore.drop_duplicates(subset=['tag', 'movieId'], inplace=True)

    # Pivoting the data frame
    df_gtagscore = df_gtagscore.pivot(index='movieId', columns='tag', values='relevance')

    # Adding release years as columns
    df_movies = pd.read_csv('data/movies.csv')
   
    # Getting and attaching the release year as a new column
    df_movies['releaseyear'] = get_releaseyear(df_movies['title'])
    
    # Normalize the release year values
    from sklearn.preprocessing import MinMaxScaler, LabelEncoder
    scaler = MinMaxScaler()
    df_movies.releaseyear = scaler.fit_transform(df_movies[['releaseyear']])
    
    # Adding genres as columns
    df_movies.genres = df_movies.genres.str.split('|')
    dummies = pd.get_dummies(df_movies.genres.apply(pd.Series).stack()).sum(level=0)
    df_movies = pd.concat([df_movies, dummies], axis=1).drop('genres', axis=1)

    # Merging the tag score dataset and the new dataset with release year and genres to a new database
    df_movies = df_movies.drop('title', axis=1)
    df_ContBaseRec = pd.merge(df_gtagscore, df_movies, how='inner', on='movieId').set_index('movieId')
    df_ContBaseRec.columns = df_ContBaseRec.columns.astype(str)
    return df_ContBaseRec

In [3]:
df = get_df()

C:\Users\HamidEsfahlani\AppData\Local\Temp\ipykernel_25196\1789884583.py:30: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df_movies.genres.apply(pd.Series).stack()).sum(level=0)


# Clustering the data

In [6]:
# Clustering the data and getting the labeled movies dataset
from data_script.Clustering import get_labeledMovies
df_labeled = get_labeledMovies(df, 18)

# getting a list of rated movies. We consider this movies as already watched

In [4]:
# List of movie IDs present in df
movie_ids_in_df = df.index.tolist()

# Filter movies in df_rating that are present in df
df_rating_filtered = df_rating[df_rating['movieId'].isin(movie_ids_in_df)]

# Group by userId and count the number of ratings
user_ratings_count = df_rating_filtered.groupby('userId')['rating'].count()

# Filter users based on the ratings count condition
filtered_users = user_ratings_count[(user_ratings_count > 150) & (user_ratings_count < 200)].index

# Filter df_rating to keep only the rows for filtered users
df_rating_final = df_rating_filtered[df_rating_filtered['userId'].isin(filtered_users)]

# Group by userId and aggregate the movieIds as a list
user_rated_movies = df_rating_final.groupby('userId')['movieId'].apply(np.array)

# Create the final DataFrame
result_df = pd.DataFrame({'RatedMovies': user_rated_movies})

result_df

,RatedMovies
userId,
1,"[2, 29, 32, 47, 50, 112, 151, 223, 253, 260, 2..."
3,"[1, 24, 32, 50, 160, 173, 175, 196, 223, 260, ..."
21,"[6, 16, 32, 50, 111, 223, 235, 260, 262, 293, ..."
29,"[2, 9, 11, 15, 16, 19, 20, 21, 22, 23, 24, 31,..."
35,"[47, 50, 60, 107, 110, 111, 153, 208, 231, 253..."
...,...
138416,"[1, 2, 10, 32, 47, 50, 69, 110, 150, 165, 180,..."
138417,"[24, 66, 111, 196, 258, 260, 316, 318, 350, 35..."
138441,"[1, 13, 48, 239, 364, 588, 594, 595, 596, 616,..."


In [26]:
result_df.iloc[1,0]

array([   1,   24,   32,   50,  160,  173,  175,  196,  223,  260,  316,
        318,  329,  337,  440,  442,  457,  480,  490,  512,  541,  589,
        593,  610,  718,  780,  788,  858,  904,  905,  919,  924,  953,
        968, 1037, 1060, 1073, 1077, 1079, 1084, 1089, 1094, 1097, 1103,
       1125, 1127, 1129, 1179, 1188, 1193, 1196, 1197, 1198, 1199, 1200,
       1206, 1208, 1210, 1213, 1214, 1215, 1219, 1220, 1221, 1222, 1225,
       1228, 1230, 1240, 1242, 1247, 1257, 1258, 1259, 1261, 1266, 1270,
       1272, 1276, 1278, 1288, 1304, 1307, 1321, 1330, 1333, 1345, 1356,
       1372, 1373, 1374, 1375, 1376, 1396, 1544, 1584, 1603, 1653, 1674,
       1676, 1721, 1762, 1779, 1810, 1831, 1876, 1882, 1909, 1917, 1921,
       2009, 2011, 2012, 2018, 2028, 2034, 2046, 2053, 2054, 2076, 2093,
       2105, 2117, 2118, 2140, 2150, 2236, 2288, 2311, 2329, 2366, 2371,
       2391, 2407, 2428, 2448, 2455, 2505, 2528, 2529, 2530, 2531, 2532,
       2533, 2541, 2551, 2567, 2571, 2574, 2613, 26

# modifying the recommendation, so it takes as input df and an array

In [30]:
def recommendation(given_movies_ids, df):
   # Get the database
   given_movies_ids = given_movies_ids.tolist()
   # Get rid of the movies not represented in the preprocessed database
   given_movies_ids = [item for item in given_movies_ids if item in df.index.tolist()]
   
   # Calculating the mean point (vector) of the given movies
   mean_point = df.loc[given_movies_ids].mean()

   ## Movie recommendation with cousin similarity
   
   from sklearn.metrics.pairwise import cosine_similarity
   # Get the relevance vector of the given movie
   given_movie_vector = mean_point.values.reshape(1, -1)
   # Calculate cosine similarity between the given movie and all other movies
   similarity_scores = cosine_similarity(df.values, given_movie_vector)
   

   ## Calculate Distances of the mean vector to the nearest neighbors
   
   # Import and fit the model
   import sklearn.neighbors
   nn = sklearn.neighbors.NearestNeighbors(n_neighbors=df.shape[0])
   nn.fit(df)
   # Getting the distances of any point in the df to mean point
   distances, indices = nn.kneighbors(mean_point.values.reshape(1, -1))
   # Normalize the data
   distances = (distances - distances.min()) / (distances.max() - distances.min())
   # Sorting the distance according to the original index of the df
   distances = distances[0][np.argsort(indices[0])]
   
   # Diversification
   score = pd.Series((similarity_scores.reshape(1, -1) - (0.5/4 * distances))[0], name="score")

   # Concatenate scores to the df
   df_scored = pd.concat([df_labeled.reset_index(names='movieId'), score.reset_index(drop=True)],
                        axis=1).set_index('movieId')

   # Remove given movies from the list of recommendation
   df_scored = df_scored.drop(given_movies_ids)
   # Removing movies that are not in the same clusters as the given movies
   given_movies_clusters = df_labeled.loc[given_movies_ids, "labels"].unique()
   df_scored = df_scored[df_scored["labels"].isin(given_movies_clusters)]
   #if the number of recommendation is set to zero, we get the score data frame. 
   return df_scored
    


In [41]:
recommendation(result_df.iloc[258,0],df).score

X does not have valid feature names, but NearestNeighbors was fitted with feature names


movieId
3         0.821889
4         0.737549
5         0.745307
6         0.719306
7         0.726804
            ...   
130578    0.645863
130840    0.693626
131013    0.724211
131168    0.660894
131170    0.735132
Name: score, Length: 9298, dtype: float64

In [26]:
result_df.RatedMovies.isna().any()

False

In [28]:
import shap
explainer = shap.KernelExplainer(recommendation, df)


Provided model function fails when applied to the provided data set.


ValueError: Input contains NaN.

In [59]:
import shap
accumulated_shap = np.zeros(df.shape[1])
i = 0
for i in range(result_df.RatedMovies.shape[0]):
    explainer = shap.KernelExplainer(recommendation, result_df.RatedMovies.iloc[i])
    shap_values = explainer.shap_values(result_df.RatedMovies.iloc[i])
    accumulated_shap += tag_contribution


IndexError: tuple index out of range

In [43]:

# Create an array to accumulate SHAP values for each tag
tag_perturbation_accumulated = pd.DataFrame(np.zeros(df.shape)).set_index(df.index)

# Iterate through each user's rated movies
for row in result_df.RatedMovies:
    # Calculate the original recommendation scores
    original_scores = recommendation(row, df).score
    
    # Iterate through each tag for perturbation analysis
    for tag_index, tag in enumerate(df.columns):
        df_perturbed = df.drop([tag], axis=1)
        tag_contribution = original_scores - (recommendation(row, df_perturbed)).score
        tag_perturbation_accumulated.iloc[:,tag_index] += tag_contribution

# Calculate average contribution for each tag
average_tag_contributions = tag_perturbation_accumulated / len(result_df)

# Normalize and visualize average tag contributions
normalized_contributions = (average_tag_contributions - average_tag_contributions.min()) / (average_tag_contributions.max() - average_tag_contributions.min())

# Print or visualize the normalized average tag contributions
print(normalized_contributions)

X does not have valid feature names, but NearestNeighbors was fitted with feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X does not have valid feature names, but NearestNeighbors was fitted with feature names
X does not have valid feature na

KeyboardInterrupt: 